In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Light Curves with Different Pipelines

Sadly, we expect *most* of the weirdest behavior to be artifacts either from systematic effects, or from data processing.

The bulk produced light curves we used to find all the anomalies were produced with an `eleanor` pipeline. Every pipeline is optimized for discovering different phenomena and can affect light curve production differently, so our first step is to recreate the interesting light curve, and see if we can eliminate as many systematic explanations as possible, or else discover the issue.

## Lightkurve

In [ ]:
import lightkurve
import eleanor
from lightkurve.correctors import DesignMatrix
from lightkurve.correctors import RegressionCorrector

In [ ]:
tic = 77474435
sector = 5
i=0
while i<10:
    try:
        print('trying')
        tpf = lightkurve.search_tesscut(f'TIC {tic}',sector=sector).download_all(cutout_size=(15,15))
        print('success')
        tpf=tpf[0]
        break
    except Exception:
        print(f'error: {Exception}')
        i+=1


Many of the sources may have bad defaults or no predefined mask, you'll probably have to create a custom aperture mask. See `lightkurve` Section 3 [here](https://docs.lightkurve.org/tutorials/2-creating-light-curves/2-1-custom-aperture-photometry.html) for guidance.

In [ ]:
custom_mask = tpf[0].create_threshold_mask(threshold=15)
tpf.plot(aperture_mask=custom_mask)

In [ ]:
lc = tpf.to_lightcurve(aperture_mask=custom_mask)
lc.plot();

This is raw flux without any correction, let's try different background subtractions to see if our mystery signal resolves itself.

### Simple Background Subtraction

In [ ]:
regressors = tpf.flux[:, ~custom_mask]
regressors.shape

In [ ]:
bkg = np.median(regressors, axis=1)
bkg -= np.percentile(bkg, 5)

npix = custom_mask.sum()
median_subtracted_lc = lc - npix * bkg

In [ ]:
fig, ax = plt.subplots(1, figsize=(17,10))
#lc.plot(ax=ax)
median_subtracted_lc.plot(ax=ax, normalize=True, label="Median Subtracted")
median_subtracted_lc.flatten().plot(ax=ax, normalize=True, label="+Flattened")

#ax.plot(lc.time.value, bkg)


### [Linear Regression](https://docs.lightkurve.org/tutorials/2-creating-light-curves/2-3-removing-scattered-light-using-regressioncorrector.html)

It may be necessary to remove scattered light. Scattered light is recognizable in TESS FFI light curves as a brightening trend in both halves of each light curve. The median background subtraction used above may account for this well enough, but if not try a linear regression or the pixel level decorrelation (PLD) corrector. If this removes the signal of interest, examine the pixel level information and discard the corrections if your signal is in fact localized (not present everywhere).

In [ ]:
# Make an aperture mask and an uncorrected light curve
uncorrected_lc = tpf.to_lightcurve(aperture_mask=custom_mask)

# Make a design matrix and pass it to a linear regression corrector
dm = DesignMatrix(tpf.flux[:, ~custom_mask], name='regressors').pca(5).append_constant()
rc = RegressionCorrector(uncorrected_lc)
corrected_ffi_lc = rc.correct(dm)

# Optional: Remove the scattered light, allowing for the large offset from scattered light
corrected_ffi_lc = uncorrected_lc - rc.model_lc + np.percentile(rc.model_lc.flux, 5)
corrected_ffi_lc.plot()

In [ ]:
pld = PLDCorrector(tpf, aperture_mask=custom_mask)
pld_corrected_lc = pld.correct(restore_trend=False, pca_components=5)
pld.diagnose();

In [ ]:
pld.diagnose_masks();

## eleanor

In [ ]:
star = eleanor.Source(tic=tic, sector=sector)

In [ ]:
data = eleanor.TargetData(star, height=15, width=15, bkg_size=31, do_psf=True, do_pca=True, regressors='corner')

Check that the aperture makes sense

In [ ]:
vis = eleanor.Visualize(data)
fig = vis.aperture_contour()

Look at the eleanor light curves, see if they match ours

In [ ]:
plt.figure(figsize=(15,5))

q = data.quality == 0

plt.plot(data.time[q], data.raw_flux[q]/np.nanmedian(data.raw_flux[q])+0.06, label="raw")
plt.plot(data.time[q], data.corr_flux[q]/np.nanmedian(data.corr_flux[q]) + 0.03, label="corr")
plt.plot(data.time[q], data.pca_flux[q]/np.nanmedian(data.pca_flux[q]), label="PCA")
plt.plot(data.time[q], data.psf_flux[q]/np.nanmedian(data.psf_flux[q]) - 0.02, label="PSF")
plt.ylabel('Normalized Flux')
plt.xlabel('Time [BJD - 2457000]')
plt.legend()

Let's check if the background is strongly related to the behaviors we're seeing

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(data.time[q], data.flux_bkg[q], 'k', label='1D postcard', linewidth=3)
plt.plot(data.time[q], data.tpf_flux_bkg[q], 'r--', label='1D TPF', linewidth=2)
plt.ylabel('Normalized Flux')
plt.xlabel('Time [BJD - 2457000]')
plt.legend();

# It's still weird!

### Check for contamination

In [ ]:
tpf.plot_pixels(corrector_func=lambda x: x.remove_nans().flatten().remove_outliers(), show_flux=True, aperture_mask=custom_mask);


If the behavior is still present, check to see if there's anything funky in the background of the TPF, like a rolling signal, or some transient that coincides with the target that could mess up a simple background subtraction

Let's watch a video of the pixels and see if anything pops out immediately (like some really bad background, a saturated source dominating the flux, cosmic rays or other nonsense).

In [ ]:
tpf.animate()

# Is it periodic? Phase fold that

Choose which light curve you'd like to work with from the workflows above, and define it as `working_lc`

In [ ]:
working_lc = median_subtracted_lc.flatten()

In [ ]:
pg = working_lc.normalize(unit='ppm').to_periodogram()
pg.plot(scale='linear');

In [ ]:

period = pg.period_at_max_power
period

In [ ]:
from ipywidgets import FloatSlider, interact

In [ ]:
def fold_lc(period):
    working_lc.fold(period).scatter(label=f'Period = {period:.3f} d');
    return
interact(fold_lc, period=FloatSlider(min=0.01, max=15.0, value=period.value, step=.001))

In [2]:
from PhaseFold import phasefold

In [ ]:
phasefold.fold(working_lc, sect=sector)